In [2]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# import libraries
import pandas as pd
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

###Extract image path

In [5]:
# move to train_images folder
%cd '/content/drive/My Drive/train_images'

/content/drive/My Drive/train_images


In [0]:
# extract all images ending with -1.jpg
globbed_files = glob.glob("*-1.jpg") #creates a list of all image files

In [0]:
# create dataframe with -1.jpg 
data = []

for jpg in globbed_files:
    data.append(os.path.basename(jpg))

In [0]:
# create dataframe from list (data)
df_filenames = pd.DataFrame(data)

In [0]:
# rename column 0 to filename
df_filenames.rename(columns = {0:'filename'}, inplace=True)

In [11]:
df_filenames.head()

""


In [0]:
# create PetID column by extracting first 10 character from filename column. 
# will be used to merge on.

df_filenames['PetID'] = df_filenames['filename'].str[:9]
df_filenames.head()

,filename,PetID
0,2d725d001-3.jpg,2d725d001
1,bf9bd91e1-1.jpg,bf9bd91e1
2,9cb525274-3.jpg,9cb525274
3,16ffedcf8-7.jpg,16ffedcf8
4,a63364c39-1.jpg,a63364c39


###Load tabular data

In [0]:
# load csv
df = pd.read_csv("train.csv")

In [0]:
#feature engineering

#replace 0 with 1 to make classes more balanced
df['AdoptionSpeed'].replace(to_replace=0, value=1, inplace=True)

#fill Description NAs with "" to prevent error in modelling later
df['Description'].fillna('This is my pet', inplace=True)

In [0]:
#train test split
train, test = train_test_split(df, test_size=0.25, random_state=42, stratify=df['AdoptionSpeed'])

In [0]:
#export train dataset
train.to_csv("train_c4_tabular.csv")

In [0]:
#export test dataset
test.to_csv("test_c4_tabular.csv")

###Create train dataset for images

In [0]:
#keep only two columns
train = train[['PetID', 'AdoptionSpeed']]

In [0]:
# merge filenames and pet_id on "PetID"
df_train = df_filenames.merge(train, on = "PetID")
df_train.head()

,filename,PetID,AdoptionSpeed
0,a63364c39-1.jpg,a63364c39,1
1,7c9e666a7-1.jpg,7c9e666a7,4
2,7d1d7a4f6-1.jpg,7d1d7a4f6,2
3,f4c27e753-1.jpg,f4c27e753,4
4,8bcb0a0e6-1.jpg,8bcb0a0e6,2


In [0]:
# drop duplicates
df_train.drop_duplicates(subset = 'PetID', inplace=True)

In [0]:
# drop PetID to match format needed for image ML. 
# format is filename, target

df_train.drop('PetID', axis = 1, inplace=True)

In [0]:
#rename column names
df_train.rename(columns = {'filename':'name', 'AdoptionSpeed':'label'}, inplace=True)
df_train.head()

In [0]:
# export file names
df_train.to_csv("train_images_c4_pp.csv")

###Create test dataset for images

In [0]:
#keep only two columns
test = test[['PetID', 'AdoptionSpeed']]

In [0]:
# merge filenames and pet_id on "PetID"
df_test = df_filenames.merge(test, on = "PetID")
df_test.head()

,filename,PetID,AdoptionSpeed
0,bf9bd91e1-1.jpg,bf9bd91e1,4
1,76024f2ed-1.jpg,76024f2ed,2
2,1de8d0099-1.jpg,1de8d0099,4
3,0a38a1059-1.jpg,0a38a1059,4
4,36b7411f9-1.jpg,36b7411f9,1


In [0]:
# drop duplicates
df_test.drop_duplicates(subset = 'PetID', inplace=True)
df_test_1 = df_test.copy()

In [0]:
# drop PetID to match format needed for image ML. 
# format is filename, target

df_test.drop('PetID', axis = 1, inplace=True)

In [0]:
#change column names
df_test.rename(columns = {'filename':'name', 'AdoptionSpeed':'label'}, inplace=True)
df_test.head()

In [0]:
# export df_test
df_test.to_csv("test_images_c4_pp.csv", index=False)